## Conceptual description

As people interact, they tend to become more alike in their beliefs, attitudes and behaviour. In "The Dissemination of Culture: A Model with Local Convergence and Global Polarization" (1997), Robert Axelrod presents an agent-based model to explain cultural diffusion. Analogous to Schelling's segregation model, the key to this conceptualization is the emergence of polarization from the interaction of individual agents. The basic premise is that the more similar an actor is to a neighbor, the more likely that that actor will adopt one of the neighbor's traits.

In the model below, this is implemented by initializing the model by filling an excel-like grid with agents with random values [0,1] for each of four traits (music, sports, favorite color and drink):

- *music* = {Rock, Jazz}
- *sports* = {Soccer, Cycling}
- *favorite color* = {Red, Blue}
- *drink* = {Tea, Coffee}

Each step, each agent (in random order) chooses a random neighbor from the 8 neighbors proportionaly to how similar it is to each of its neighbors, and adopts one randomly selected differing trait from this neighbor. 

To visualize the model, the four traits are transformed into 'RGBA' (Red-Green-Blue-Alpha) values; i.e. a color and an opacity. The visualizations below show the clusters of homogeneity being formed.


In [1]:
import collections
import random

import numpy as np

from mesa import Model, Agent
import mesa.time as time
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

class CulturalDiff(Model):
    """
    Model class for the Schelling segregation model.
    
    Parameters
    ----------
    height : int
             height of grid
    width : int
            height of grid
    seed : int
            random seed
    
    Attributes
    ----------
    height : int
    width : int
    density : float
    schedule : RandomActivation instance
    grid : SingleGrid instance
    
    """

    def __init__(self, height=20, width=20, seed=None):
        super().__init__(seed=seed)
        height = height
        width = width

        schedule = time.BaseScheduler(self)
        grid = SingleGrid(width, height, torus=True)
        self.datacollector = DataCollector(model_reporters={'diversity':calculate_nr_of_cultures})

        # Fill grid with agents with random traits
        
        # Note that this implementation does not guarantee some set distribution of traits. 
        # Therefore, examining the effect of minorities etc is not facilitated.
        for cell in self.grid.coord_iter():
            profile = np.asarray([self.random.choice([0,1]) for _ in range(4)])
            
            agent = CulturalDiffAgent(cell, self, profile)
            self.grid.position_agent(agent, cell)
            self.schedule.add(agent)

    def step(self):
        """
        Run one step of the model.
        """
        self.datacollector.collect(self)
        self.schedule.step


class CulturalDiffAgent(Agent):
    """
    Schelling segregation agent
    
    Parameters
    ----------
    pos : tuple of 2 ints
          the x,y coordinates in the grid
    model : Model instance
    profile : ndarray
    
    """

    def __init__(self, pos, model, profile):
        super().__init__(pos, model)
        self.pos = pos
        self.profile = profile
        
    def step(self):
        
        #For each neighbor, calculate the similarity
        neighbor_similarity_dict = {}
        for neighbor in self.model.grid.neighbor_iter(self.pos, moore=True):
            neighbor_similarity = np.sum(self.profile==neighbor.profile)
            neighbor_similarity_dict[neighbor] = neighbor_similarity
        
        # Proportional to this similarity, pick a 'random' neighbor to interact with
        neighbor_to_interact = random.choices(list(neighbor_similarity_dict.keys()),
                                                   weights=neighbor_similarity_dict.values())[0]
        
        # Select a trait that differs between the selected neighbor and self and change that trait in self
        # we are using some numpy boolean indexing to make this short and easy
        not_same_features = self.profile != neighbor_to_interact.profile
        
        if np.any(not_same_features)
            index_for_trait = self.random.choice(np.nonzero(not_same_features)[0])
            self.profile[index_for_trait] = neighbor_to_interact.profile[index_for_trait]


def traits_to_color(profile):  
    """ Converts the traits of an agent to a list of RGBA values"""
    color = profile.copy().astype(float)
    if color[-1]==0:
        color[-1] = 0.2
    return color

def calculate_nr_of_cultures(model):
    diversity = collections.defaultdict(int)
    for (cell, i, j) in model.grid.coord_iter():
        if cell:
            diversity[tuple(cell.profile)] += 1

    return len(diversity.keys())

SyntaxError: invalid syntax (<ipython-input-1-92a5ed848800>, line 94)

# Visualization

## Static images
Visualization of this model are static images. A visualization after initialization, after 20 steps, after 50 steps, and after 200 steps is presented.

### After initialization

In [ ]:
model = CulturalDiff(seed=123456789)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import pandas as pd


def plot_model(model, ax):
    grid = np.zeros((model.height, model.width, 4))

    for (cell, i, j) in model.grid.coord_iter():
        color = [0,0,0,0] #in case not every cell is filled, the default colour is white
        if cell is not None:
            color = traits_to_color(cell.profile)
        grid[i,j] = color          
    plt.imshow(grid)
    
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 20 steps

In [ ]:
for i in range(20):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 50 steps

In [ ]:
for i in range(30):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 200 steps

In [ ]:
for i in range(150):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()